# Hypothesis
We begin this project comparing the datasets of Chicago life expectancies with USA life expectancies with the hypothesis that the average Chicagoan's life expectancy is lower than the average US denizen's life expectancy.

In [1]:
import pandas as pd,numpy as np, matplotlib.pyplot as plt, matplotlib.ticker as mtick, seaborn as sns
from sodapy import Socrata
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.transform import factor_cmap
from bokeh.models import ColumnDataSource
from bokeh.palettes import Plasma5

%matplotlib inline
output_notebook()

Loading BokehJS ...

# Data Acquisition
We acquire our data on life expectancy based on race from two different types of data sources. The data regarding the city of Chicago is obtained through the Socrata api and then put into the dfc(hicago) dataframe. The data regarding the USA as a whole is obtained in the form of a relational csv from the CDC website directly and then put into the dfu(sa) dataframe.

In [2]:
host = Socrata("data.cityofchicago.org", None)
chicago_data = host.get("3qdj-cqb8")
dfc = pd.DataFrame.from_records(chicago_data)
dfu = pd.read_csv("https://data.cdc.gov/api/views/w9j2-ggv5/rows.csv")

In [3]:
dfc.head(3).append(dfc.tail(3))

,race_ethnicity,sex,_1990_life_expectancy,_1990_lower_95_ci,_1990_upper_95_ci,_2000_life_expectancy,_2000_lower_95_ci,_2000_upper_95_ci,_2010_life_expectancy,_2010_lower_95_ci,_2010_upper_95_ci
0,All,All,70.5,70.3,70.7,73.9,73.8,74.1,77.8,77.6,78.0
1,Hispanic,All,82.6,82.0,83.1,81.4,81.0,81.9,84.7,84.4,85.1
2,Non-Hispanic Black,All,65.9,65.7,66.2,68.7,68.4,68.9,72.4,72.1,72.7
9,Hispanic,Male,77.9,77.2,78.7,78.8,78.2,79.4,82.0,81.4,82.4
10,Non-Hispanic Black,Male,60.2,59.8,60.6,63.6,63.3,64.0,67.8,67.4,68.2
11,Non-Hispanic White,Male,69.0,68.6,69.3,72.6,72.3,73.0,76.5,76.2,76.9


In [4]:
dfu

,Year,Race,Sex,Average Life Expectancy (Years),Age-adjusted Death Rate
0,1900,All Races,Both Sexes,47.3,2518.0
1,1901,All Races,Both Sexes,49.1,2473.1
2,1902,All Races,Both Sexes,51.5,2301.3
3,1903,All Races,Both Sexes,50.5,2379.0
4,1904,All Races,Both Sexes,47.6,2502.5
...,...,...,...,...,...
1057,2013,White,Male,76.7,859.2
1058,2014,White,Male,76.7,853.4
1059,2015,White,Male,76.6,861.9
1060,2016,White,Male,76.4,858.9


# Data Clean-Up and Transformation

Since we want to make a direct comparison between just the life expectancy based on race in Chicago to the USA, we have a ways to go in data processing. The end goal is merging the two dataframes into one dataframe that has a new region column with the variables Chicago and USA. To do this we have to standarize the two dataframes. This involves: Converting the data in dfc from wide to long so that year becomes its own column; Getting rid of columns from both frames that aren't average life expectancy, year, race, or sex; Changing the names of the categorical variables of dfc to match dfu; And changing the column names to be the same.

### Dropping Unused Columns

We use pandas drop function to get rid of columns that we won't be using in our analysis.

In [5]:
dfc.drop(dfc.filter(regex='.*95_ci$').columns, axis=1, inplace = True)
dfu.drop(['Age-adjusted Death Rate'], axis = 1, inplace = True)

In [6]:
dfu

,Year,Race,Sex,Average Life Expectancy (Years)
0,1900,All Races,Both Sexes,47.3
1,1901,All Races,Both Sexes,49.1
2,1902,All Races,Both Sexes,51.5
3,1903,All Races,Both Sexes,50.5
4,1904,All Races,Both Sexes,47.6
...,...,...,...,...
1057,2013,White,Male,76.7
1058,2014,White,Male,76.7
1059,2015,White,Male,76.6
1060,2016,White,Male,76.4


In [7]:
dfc.head()

,race_ethnicity,sex,_1990_life_expectancy,_2000_life_expectancy,_2010_life_expectancy
0,All,All,70.5,73.9,77.8
1,Hispanic,All,82.6,81.4,84.7
2,Non-Hispanic Black,All,65.9,68.7,72.4
3,Non-Hispanic White,All,73.2,76.0,79.2
4,All,Female,75.1,77.5,79.8


### Converting from Wide to Long

We use pandas melt function to restructure and transform our data so that we now have a Year column to match dfu. We also change dfu's Year to str from int64 using astype to match the type in the DFC Year column. Average life expectancy is also currently an object type so we have to change it to float64 to match dfu as well.

In [8]:
dfc = dfc.melt(id_vars=["race_ethnicity","sex"], var_name = "Year", value_name = "Life Expectancy")

In [9]:
dfc['Year'] = dfc['Year'].str.replace(r'\D+', '')
dfu['Year'] = dfu['Year'].astype('str')
dfc['Life Expectancy'] = dfc['Life Expectancy'].astype('float64')

In [10]:
dfc.head(3).append(dfc.tail(3))

,race_ethnicity,sex,Year,Life Expectancy
0,All,All,1990,70.5
1,Hispanic,All,1990,82.6
2,Non-Hispanic Black,All,1990,65.9
33,Hispanic,Male,2010,82.0
34,Non-Hispanic Black,Male,2010,67.8
35,Non-Hispanic White,Male,2010,76.5


### Reordering Columns

I used the pandas reindex feature which we did not cover in class in order to have the dfc columns in the same order as dfu so I can properly merge the dataframes into one.

In [11]:
dfc = dfc.reindex(columns = ['Year','race_ethnicity','sex','Life Expectancy'])

In [12]:
dfc.head()

,Year,race_ethnicity,sex,Life Expectancy
0,1990,All,All,70.5
1,1990,Hispanic,All,82.6
2,1990,Non-Hispanic Black,All,65.9
3,1990,Non-Hispanic White,All,73.2
4,1990,All,Female,75.1


### Standarizing Column Names
In order to properly concatenate our two dataframes we want to make sure that our column names are the same between both dataframes. We will change the dfc column names to match the dfu ones for the columns that are not matching already. We can do this quite simply utilizing pandas' rename function.

In [13]:
dfc.rename(columns={'race_ethnicity':'Race','sex':'Sex'},inplace=True)
dfu.rename(columns={'Average Life Expectancy (Years)':'Life Expectancy'}, inplace=True)

In [14]:
dfc.head()

,Year,Race,Sex,Life Expectancy
0,1990,All,All,70.5
1,1990,Hispanic,All,82.6
2,1990,Non-Hispanic Black,All,65.9
3,1990,Non-Hispanic White,All,73.2
4,1990,All,Female,75.1


### Standarizing Values
Our categorical variables of race and sex have different values which we should change into one value for proper analysis. All will be used in place of Both Sexes and All Races in dfu. While in dfc, we will change Non-Hispanic Black and White to just Black and White. However, it must be kept in mind that this Chicago data will still be for non-hispanic black and white people which is distinctly different than just black and white without hispanic being taken into consideration.

In [15]:
dfu['Race'] = dfu['Race'].str.replace("All Races","All")
dfu['Sex'] = dfu['Sex'].str.replace("Both Sexes","All")
dfc['Race'] = dfc['Race'].str.replace('Non-Hispanic','').str.strip()

In [16]:
dfu.head(2).append(dfu.tail(2))

,Year,Race,Sex,Life Expectancy
0,1900,All,All,47.3
1,1901,All,All,49.1
1060,2016,White,Male,76.4
1061,2017,White,Male,76.4


In [17]:
dfc.head(4)

,Year,Race,Sex,Life Expectancy
0,1990,All,All,70.5
1,1990,Hispanic,All,82.6
2,1990,Black,All,65.9
3,1990,White,All,73.2


### Adding a Region Category

Our dataframes are almost ready to be merged, however in order to compare Chicago data and US data from within a single dataframe we need a way to identify the data from each region. An easy way to do this is to add a category for each dataframe that consists of a simple categorical variable of the region, USA or CHI.

In [18]:
dfc['Region'] = 'CHI'
dfu['Region'] = 'USA'

In [19]:
dfc.head()

,Year,Race,Sex,Life Expectancy,Region
0,1990,All,All,70.5,CHI
1,1990,Hispanic,All,82.6,CHI
2,1990,Black,All,65.9,CHI
3,1990,White,All,73.2,CHI
4,1990,All,Female,75.1,CHI


### Merging the Dataframes
Our two dataframes have finally been appropriately preprocessed and are ready to be merged together using pandas' concat function into the dataframe dfa(ll). Since this is intended as a direct comparison between the two regions, we are merging only the rows that have years in both dataframes which is the years: 1990, 2000, and 2010. 

In [20]:
dfa = pd.concat([dfu.loc[dfu['Year'].isin(['1990','2000','2010'])],dfc], ignore_index=True)

In [21]:
dfa

,Year,Race,Sex,Life Expectancy,Region
0,1990,All,All,75.4,USA
1,2000,All,All,76.8,USA
2,2010,All,All,78.7,USA
3,1990,All,Female,78.8,USA
4,2000,All,Female,79.7,USA
...,...,...,...,...,...
58,2010,White,Female,81.8,CHI
59,2010,All,Male,73.6,CHI
60,2010,Hispanic,Male,82.0,CHI
61,2010,Black,Male,67.8,CHI


# Data Analysis and Presentation

The data is all ready to be analyzed now. We're going to want to group and aggregate the data for summary statistics. Crete graphics that describe the data. Create a graphic that supports a conclusion and finally go over a statistical analysis that supports the conclusion as well.

### Data Description

We begin with describing our freshly repackaged dataset. Usually the pandas describe function would be of some help here, however with most of our variables being categorical it only tells us about the singular quantitative variable of life expectency. Through this we are able to see that the lowest recorded life expectancy within our dataset was 60.2, the mean was 75.44, and the max life expectancy was 87.6.

In [22]:
dfa.describe()

,Life Expectancy
count,63.000000
mean,75.442857
std,5.600193
min,60.200000
25%,72.100000
50%,76.100000
75%,79.050000
max,87.600000


### Data Aggregation and Summary Analysis
The description doesn't tell us that much, so we aggregate and group the data into two groupings to provide a summary view of the data itself. We also plot these summary views with bokeh being a new feature not covered in class. Utilizing these sumamry views we are able to provide some basic statistical analyses. We are able to see that life expectancy typically increases over year, that women live longer, and beyond the hispanic race Chicagoans seem to have a lower life expectancy than the rest of the nation. 

In [23]:
dfa.query('''Sex == 'All' ''').groupby(['Year','Race','Region']).mean()

Life Expectancy
Year Race     Region                 
1990 All      CHI                70.5
              USA                75.4
     Black    CHI                65.9
              USA                69.1
     Hispanic CHI                82.6
     White    CHI                73.2
              USA                76.1
2000 All      CHI                73.9
              USA                76.8
     Black    CHI                68.7
              USA                71.8
     Hispanic CHI                81.4
     White    CHI                76.0
              USA                77.3
2010 All      CHI                77.8
              USA                78.7
     Black    CHI                72.4
              USA                75.1
     Hispanic CHI                84.7
     White    CHI                79.2
              USA                78.9

In [24]:
group = dfa.query('''Sex == 'All' ''').groupby(['Year','Race','Region'])
source = ColumnDataSource(group)
index_cmap = factor_cmap('Year_Race_Region', palette=Plasma5, factors=sorted(dfa.Year.unique()),end =1)

p = figure(plot_width = 800, plot_height=400, x_range=group, title="Life Expectancy By Year, Race, and Region",
           toolbar_location=None, tooltips = [("Expectancy","@{Life Expectancy_mean}")])

p.vbar(x='Year_Race_Region', top='Life Expectancy_mean', width=1, source=group,
       line_color="white", fill_color=index_cmap)

p.y_range.start = 0
p.x_range.range_padding = 0.05
p.xgrid.grid_line_color = None
p.xaxis.axis_label = "Region Grouped By Race and then Year"
p.xaxis.major_label_orientation = 1.2
p.outline_line_color = None

show(p)

In [25]:
dfa.groupby(['Sex','Race','Region']).mean()

Life Expectancy
Sex    Race     Region                 
All    All      CHI           74.066667
                USA           76.966667
       Black    CHI           69.000000
                USA           72.000000
       Hispanic CHI           82.900000
       White    CHI           76.133333
                USA           77.433333
Female All      CHI           77.466667
                USA           79.833333
       Black    CHI           73.733333
                USA           75.566667
       Hispanic CHI           86.500000
       White    CHI           79.533333
                USA           80.200000
Male   All      CHI           69.933333
                USA           74.100000
       Black    CHI           63.866667
                USA           68.166667
       Hispanic CHI           79.566667
       White    CHI           72.700000
                USA           74.633333

In [26]:
group = dfa.groupby(['Sex','Race','Region'])
source = ColumnDataSource(group)
index_cmap = factor_cmap('Sex_Race_Region', palette=Plasma5, factors=sorted(dfa.Sex.unique()),end =1)

p = figure(plot_width = 800, plot_height=400, x_range=group, title="Life Expectancy By Sex, Race, and Region",
           toolbar_location=None, tooltips = [("Expectancy","@{Life Expectancy_mean}")])

p.vbar(x='Sex_Race_Region', top='Life Expectancy_mean', width=1, source=group,
       line_color="white", fill_color=index_cmap)

p.y_range.start = 0
p.x_range.range_padding = 0.05
p.xgrid.grid_line_color = None
p.xaxis.axis_label = "Sex Grouped By Race and then Year"
p.xaxis.major_label_orientation = 1.2
p.outline_line_color = None

show(p)

### Hypothesis Validation
Our summary data does seem to indicate that our hypothesis that Chicagoans have lower life expectancies than the rest of the USA may have some validity to it. However, we want to take a closer look at that claim by creating graphics and tables that specifically focus on validating this hypothesis.

In [27]:
dfa.query('''Race == 'All' and Sex == 'All' ''').groupby(['Year','Region']).mean()

Life Expectancy
Year Region                 
1990 CHI                70.5
     USA                75.4
2000 CHI                73.9
     USA                76.8
2010 CHI                77.8
     USA                78.7

In [28]:
group = dfa.query('''Race == 'All' and Sex == 'All' ''').groupby(['Year','Region'])
source = ColumnDataSource(group)
index_cmap = factor_cmap('Year_Region', palette=Plasma5, factors=sorted(dfa.Year.unique()),end =1)

p = figure(plot_width = 800, plot_height=400, x_range=group, title="Life Expectancy By Year and Region",
           toolbar_location=None, tooltips = [("Expectancy","@{Life Expectancy_mean}")])

p.vbar(x='Year_Region', top='Life Expectancy_mean', width=1, source=group,
       line_color="white", fill_color=index_cmap)

p.y_range.start = 0
p.x_range.range_padding = 0.05
p.xgrid.grid_line_color = None
p.xaxis.axis_label = "Year Grouped By Region"
p.xaxis.major_label_orientation = 1.2
p.outline_line_color = None

show(p)

In [29]:
dfa.query('''Race == 'All' and Sex == 'All' ''').groupby(['Region']).mean()

,Life Expectancy
Region,
CHI,74.066667
USA,76.966667


### Concluding Statistical Analysis

We create the life expectancy by year and region graph that puts to the side the impact of sex and race as variables by selecting only rows where it is based on all race and sex. The only variables that we have to worry about now are time and region. Every year it can be seen that the average Chicagoan's life expectancy is indeed less than the USA average life expectancy. This supports our hypothesis. The table that puts to the side every variable besides region to directly compare the average life expectancy of a Chicagoan from 1990-2010 with the average life expectancy of the average US citizen during the same time period also supports our hypothesis. As the Chicagoan life expectancy is lower than the US life expectancy. Although, the data does support our hypothesis, it should be noted that chicagoan life expectancy is catching up to US life expectancy if you look at the life expectancy by year and region graph.